<a href="https://colab.research.google.com/github/your-repo/magpie/blob/main/colab_6domains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧮 Magpie: 6ドメイン数学データセット生成 (Google Colab版)

このノートブックは、DeepSeek R1を使用して6つの数学ドメイン別データセットを生成し、統合・シャッフルするGoogle Colab版です。

## 🎯 対応ドメイン
1. **Algebra** (代数学): 方程式、多項式、関数
2. **Applied Mathematics** (応用数学): 微分方程式、最適化
3. **Calculus** (微積分学): 微積分、極限、級数
4. **Discrete Mathematics** (離散数学): 組合せ、グラフ理論
5. **Geometry** (幾何学): 解析幾何、空間図形
6. **Number Theory** (数論): 素数、合同式、暗号応用

## ⚠️ 重要事項
- **GPU必須**: A100推奨、T4でも動作可能（軽量モード）
- **実行時間**: 各ドメイン10-50問程度で30分-2時間
- **メモリ制限**: Colabの制限に応じて問題数を調整

## ⚙️ ユーザー設定

In [ ]:
# ===== ユーザー設定 =====

# 基本設定
DATASET_NAME = "HLE_6Domains_Math"  # データセット名
PROBLEMS_PER_DOMAIN = 20  # 各ドメインの問題数（Colab用に少なめ）
OUTPUT_DIR = "/content/magpie_6domains"  # 出力ディレクトリ

# モデル設定（GPU能力に応じて選択）
USE_LIGHTWEIGHT_MODEL = True  # TrueでQwen2.5-3B、FalseでDeepSeek R1

if USE_LIGHTWEIGHT_MODEL:
    MODEL_PATH = "Qwen/Qwen2.5-3B-Instruct"  # 軽量モデル（T4 GPU対応）
    TENSOR_PARALLEL = 1
    GPU_MEMORY_UTIL = 0.80
    BATCH_SIZE = 10
else:
    MODEL_PATH = "deepseek-ai/DeepSeek-R1"  # フルモデル（A100推奨）
    TENSOR_PARALLEL = 2  # Colabの制限に合わせて調整
    GPU_MEMORY_UTIL = 0.90
    BATCH_SIZE = 5

# 生成パラメータ
INSTRUCTION_TEMP = 1.2
INSTRUCTION_TOP_P = 1.0
RESPONSE_TEMP = 0.1
RESPONSE_TOP_P = 1.0

# 対象ドメイン
DOMAINS = [
    "algebra",
    "applied-mathematics", 
    "calculus",
    "discrete-mathematics",
    "geometry",
    "number-theory"
]

print(f"📊 設定完了:")
print(f"  データセット名: {DATASET_NAME}")
print(f"  各ドメイン問題数: {PROBLEMS_PER_DOMAIN}")
print(f"  総問題数: {len(DOMAINS) * PROBLEMS_PER_DOMAIN}")
print(f"  使用モデル: {MODEL_PATH}")
print(f"  軽量モード: {'有効' if USE_LIGHTWEIGHT_MODEL else '無効'}")
print(f"  出力先: {OUTPUT_DIR}")

## 🚀 環境セットアップ

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"CUDA利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# リポジトリクローンと依存関係インストール
!git clone https://github.com/your-repo/magpie.git
%cd magpie

# 必要パッケージのインストール
!pip install -q vllm transformers torch accelerate
!pip install -q datasets sentencepiece tiktoken
!pip install -q numpy pandas tqdm matplotlib

# 出力ディレクトリ作成
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ 環境セットアップ完了: {OUTPUT_DIR}")

In [ ]:
# Hugging Face認証（DeepSeek R1使用時必要）
if not USE_LIGHTWEIGHT_MODEL:
    from huggingface_hub import login
    
    print("DeepSeek R1を使用するためにHugging Faceトークンが必要です")
    print("https://huggingface.co/settings/tokens でトークンを取得してください")
    
    # 手動でトークンを入力
    # login()  # 対話的ログイン
    
    print("⚠️ 上記のlogin()のコメントアウトを外して実行してください")
else:
    print("✅ 軽量モデル使用のため認証不要")

## 🎯 Step 1: 6ドメイン別データ生成

In [ ]:
import subprocess
import json
import time
from datetime import datetime

def run_domain_generation(domain, model_path, problems, timestamp):
    """単一ドメインのデータ生成"""
    print(f"\n🔄 {domain} ドメイン生成開始...")
    
    # 問題生成
    ins_cmd = [
        "python", "exp/gen_ins.py",
        "--model_path", model_path,
        "--total_prompts", str(problems),
        "--temperature", str(INSTRUCTION_TEMP),
        "--top_p", str(INSTRUCTION_TOP_P),
        "--tensor_parallel_size", str(TENSOR_PARALLEL),
        "--gpu_memory_utilization", str(GPU_MEMORY_UTIL),
        "--control_tasks", "math",
        "--domain", domain,
        "--n", str(BATCH_SIZE),
        "--job_name", f"Colab-{domain}",
        "--timestamp", str(timestamp),
        "--max_tokens", "3072",
        "--max_model_len", "8192"
    ]
    
    try:
        result = subprocess.run(ins_cmd, capture_output=True, text=True, check=True)
        print(f"✅ {domain} 問題生成完了")
    except subprocess.CalledProcessError as e:
        print(f"❌ {domain} 問題生成エラー: {e.stderr[:500]}")
        return False
    
    # 生成されたファイルを確認
    model_name = model_path.split("/")[-1]
    ins_file = f"data/Colab-{domain}/Magpie_{model_name}_{problems}_{timestamp}_ins.json"
    
    if not os.path.exists(ins_file):
        print(f"❌ {domain} 問題ファイルが見つかりません: {ins_file}")
        return False
    
    # 解答生成
    res_cmd = [
        "python", "exp/gen_res.py",
        "--model_path", model_path,
        "--input_file", ins_file,
        "--temperature", str(RESPONSE_TEMP),
        "--top_p", str(RESPONSE_TOP_P),
        "--tensor_parallel_size", str(TENSOR_PARALLEL),
        "--gpu_memory_utilization", str(GPU_MEMORY_UTIL),
        "--batch_size", str(BATCH_SIZE),
        "--repetition_penalty", "1.0",
        "--use_tokenizer_template",
        "--offline",
        "--max_tokens", "4096"
    ]
    
    try:
        result = subprocess.run(res_cmd, capture_output=True, text=True, check=True)
        print(f"✅ {domain} 解答生成完了")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {domain} 解答生成エラー: {e.stderr[:500]}")
        return False

# 生成実行
timestamp = int(time.time())
success_domains = []
failed_domains = []

print(f"🚀 6ドメインデータ生成開始")
print(f"📊 設定: {MODEL_PATH}, 各{PROBLEMS_PER_DOMAIN}問題")

for i, domain in enumerate(DOMAINS):
    print(f"\n📈 進捗: {i+1}/{len(DOMAINS)} - {domain}")
    
    if run_domain_generation(domain, MODEL_PATH, PROBLEMS_PER_DOMAIN, timestamp):
        success_domains.append(domain)
        print(f"✅ {domain} 完了")
    else:
        failed_domains.append(domain)
        print(f"❌ {domain} 失敗")
    
    # GPU cooldown
    if i < len(DOMAINS) - 1:
        print("⏸️ GPU cooldown...")
        time.sleep(10)

print(f"\n📊 生成結果:")
print(f"  成功: {len(success_domains)}/{len(DOMAINS)} ドメイン")
print(f"  成功ドメイン: {success_domains}")
if failed_domains:
    print(f"  失敗ドメイン: {failed_domains}")

## 🔄 Step 2: データ統合・シャッフル

In [ ]:
import json
import random
import glob
from pathlib import Path

def find_generated_files():
    """生成されたファイルを自動検出"""
    domain_files = {}
    model_name = MODEL_PATH.split("/")[-1]
    
    for domain in success_domains:
        pattern = f"data/Colab-{domain}/Magpie_{model_name}_*_ins_res.json"
        files = glob.glob(pattern)
        
        if files:
            domain_files[domain] = files[0]
            print(f"📁 {domain}: {files[0]}")
        else:
            print(f"⚠️ {domain}: ファイルが見つかりません")
    
    return domain_files

def merge_and_shuffle_colab(domain_files):
    """Colab版データ統合・シャッフル"""
    all_data = []
    domain_stats = {}
    
    print("📊 ドメイン別データ読み込み中...")
    
    for domain, filepath in domain_files.items():
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                domain_data = json.load(f)
            
            # ドメインメタデータ追加
            for item in domain_data:
                item['domain'] = domain
                item['source'] = 'colab-magpie'
                item['dataset_version'] = '1.0'
            
            all_data.extend(domain_data)
            domain_stats[domain] = len(domain_data)
            
            print(f"  ✅ {domain}: {len(domain_data)}問題")
            
        except Exception as e:
            print(f"  ❌ {domain}: エラー - {e}")
            continue
    
    # 統計表示
    total_problems = sum(domain_stats.values())
    print(f"\n📈 統計:")
    for domain, count in domain_stats.items():
        percentage = (count / total_problems) * 100 if total_problems > 0 else 0
        print(f"  {domain}: {count}問題 ({percentage:.1f}%)")
    print(f"  合計: {total_problems}問題")
    
    # シャッフル
    print("\n🔀 データシャッフル中...")
    random.seed(42)  # 再現性のため
    random.shuffle(all_data)
    
    return all_data, domain_stats

def create_sharegpt_format(data):
    """ShareGPT形式に変換"""
    sharegpt_data = []
    
    for i, item in enumerate(data):
        sharegpt_entry = {
            "conversation_id": f"colab-magpie-{i}",
            "domain": item.get('domain', 'unknown'),
            "source": item.get('source', 'colab-magpie'),
            "conversations": [
                {"from": "human", "value": item['instruction']},
                {"from": "gpt", "value": item['response']}
            ],
            "gen_input_configs": item.get('gen_input_configs', {}),
            "gen_response_configs": item.get('gen_response_configs', {}),
            "created": item.get('created', ''),
            "id": item.get('id', i)
        }
        sharegpt_data.append(sharegpt_entry)
    
    return sharegpt_data

# 統合実行
if success_domains:
    print("🔄 データ統合・シャッフル開始")
    
    domain_files = find_generated_files()
    
    if domain_files:
        merged_data, stats = merge_and_shuffle_colab(domain_files)
        
        # 統合ファイル保存
        timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
        merged_file = f"{OUTPUT_DIR}/{DATASET_NAME}_{len(merged_data)}_{timestamp_str}.json"
        
        with open(merged_file, 'w', encoding='utf-8') as f:
            json.dump(merged_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ 統合データ保存: {merged_file}")
        
        # ShareGPT形式
        sharegpt_data = create_sharegpt_format(merged_data)
        sharegpt_file = f"{OUTPUT_DIR}/{DATASET_NAME}_{len(merged_data)}_{timestamp_str}_sharegpt.jsonl"
        
        with open(sharegpt_file, 'w', encoding='utf-8') as f:
            for item in sharegpt_data:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        print(f"✅ ShareGPT形式保存: {sharegpt_file}")
        
        # サンプル表示
        if merged_data:
            print("\n📝 サンプルデータ:")
            sample = merged_data[0]
            print(f"ドメイン: {sample['domain']}")
            print(f"問題: {sample['instruction'][:100]}...")
            print(f"解答: {sample['response'][:100]}...")
    
    else:
        print("❌ 統合可能なファイルが見つかりません")
else:
    print("⚠️ 成功したドメインがありません。Step 1を確認してください。")

## 📊 Step 3: データ品質分析

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

def analyze_dataset_quality(data):
    """データセット品質分析"""
    analysis = {
        "total_samples": len(data),
        "domains": Counter(),
        "instruction_lengths": [],
        "response_lengths": [],
        "math_keywords": 0,
        "reasoning_patterns": 0
    }
    
    math_keywords = ['equation', 'solve', 'calculate', 'theorem', 'proof', 
                     '方程式', '計算', '定理', '証明', 'derivative', 'integral']
    reasoning_patterns = ['step', 'first', 'then', 'therefore', 'because',
                         'ステップ', 'まず', 'そして', 'したがって']
    
    for item in data:
        domain = item.get('domain', 'unknown')
        instruction = item.get('instruction', '')
        response = item.get('response', '')
        
        analysis['domains'][domain] += 1
        analysis['instruction_lengths'].append(len(instruction))
        analysis['response_lengths'].append(len(response))
        
        # 数学キーワード検出
        text = (instruction + ' ' + response).lower()
        if any(keyword in text for keyword in math_keywords):
            analysis['math_keywords'] += 1
        
        # 推論パターン検出
        if any(pattern in response.lower() for pattern in reasoning_patterns):
            analysis['reasoning_patterns'] += 1
    
    return analysis

def visualize_analysis(analysis):
    """分析結果の可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # ドメイン分布
    domains = list(analysis['domains'].keys())
    counts = list(analysis['domains'].values())
    
    axes[0,0].pie(counts, labels=domains, autopct='%1.1f%%')
    axes[0,0].set_title('ドメイン分布')
    
    # 問題文長分布
    axes[0,1].hist(analysis['instruction_lengths'], bins=20, alpha=0.7)
    axes[0,1].set_title('問題文長分布')
    axes[0,1].set_xlabel('文字数')
    axes[0,1].set_ylabel('頻度')
    
    # 解答長分布
    axes[1,0].hist(analysis['response_lengths'], bins=20, alpha=0.7, color='orange')
    axes[1,0].set_title('解答長分布')
    axes[1,0].set_xlabel('文字数')
    axes[1,0].set_ylabel('頻度')
    
    # 品質指標
    total = analysis['total_samples']
    metrics = ['数学キーワード', '推論パターン']
    values = [analysis['math_keywords']/total*100, analysis['reasoning_patterns']/total*100]
    
    axes[1,1].bar(metrics, values)
    axes[1,1].set_title('品質指標 (%)')
    axes[1,1].set_ylabel('含有率 (%)')
    
    plt.tight_layout()
    plt.savefig(f'{OUTPUT_DIR}/quality_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

# 品質分析実行
if 'merged_data' in locals() and merged_data:
    print("📊 データセット品質分析中...")
    
    analysis = analyze_dataset_quality(merged_data)
    
    print("\n📈 品質分析結果:")
    print(f"  総サンプル数: {analysis['total_samples']}")
    print(f"  平均問題文長: {np.mean(analysis['instruction_lengths']):.1f} 文字")
    print(f"  平均解答長: {np.mean(analysis['response_lengths']):.1f} 文字")
    print(f"  数学キーワード含有率: {analysis['math_keywords']/analysis['total_samples']*100:.1f}%")
    print(f"  推論パターン含有率: {analysis['reasoning_patterns']/analysis['total_samples']*100:.1f}%")
    
    print("\n📊 ドメイン別統計:")
    for domain, count in analysis['domains'].items():
        percentage = count / analysis['total_samples'] * 100
        print(f"  {domain}: {count}問題 ({percentage:.1f}%)")
    
    # 可視化
    visualize_analysis(analysis)
    
    # 分析結果保存
    analysis_file = f"{OUTPUT_DIR}/quality_analysis.json"
    analysis_serializable = {
        k: (dict(v) if isinstance(v, Counter) else v) 
        for k, v in analysis.items()
    }
    
    with open(analysis_file, 'w', encoding='utf-8') as f:
        json.dump(analysis_serializable, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 分析結果保存: {analysis_file}")

else:
    print("⚠️ 分析対象データがありません。Step 2を確認してください。")

## 📥 Step 4: ファイルダウンロード

In [ ]:
from google.colab import files
import zipfile

# 生成ファイル一覧
output_files = []
for file_path in Path(OUTPUT_DIR).glob('*'):
    if file_path.is_file():
        output_files.append(str(file_path))

print("📁 生成されたファイル:")
for file_path in output_files:
    filename = os.path.basename(file_path)
    size = os.path.getsize(file_path) / 1024  # KB
    print(f"  📄 {filename} ({size:.1f} KB)")

if output_files:
    # ZIPファイル作成
    zip_file = f"{OUTPUT_DIR}/{DATASET_NAME}_complete.zip"
    
    with zipfile.ZipFile(zip_file, 'w') as zipf:
        for file_path in output_files:
            arcname = os.path.basename(file_path)
            zipf.write(file_path, arcname)
    
    print(f"\n📦 統合ZIPファイル作成: {zip_file}")
    print(f"📊 ZIPサイズ: {os.path.getsize(zip_file) / 1024:.1f} KB")
    
    # ダウンロード
    print("\n📥 ファイルダウンロード中...")
    files.download(zip_file)
    
    print("✅ ダウンロード完了！")
    
    # 個別ダウンロードオプション
    print("\n💡 個別ファイルダウンロード:")
    print("以下のコードで個別にダウンロードできます:")
    print("```python")
    for file_path in output_files:
        if file_path.endswith('.json') or file_path.endswith('.jsonl'):
            print(f"# files.download('{file_path}')")
    print("```")

else:
    print("⚠️ ダウンロード可能なファイルがありません")

## 🎉 完了サマリー

### 📊 生成結果
- **6ドメイン数学データセット**: 代数学、応用数学、微積分学、離散数学、幾何学、数論
- **統合・シャッフル済み**: ドメインバランス保持
- **ShareGPT互換**: 機械学習フレームワーク対応

### 📁 出力ファイル
- `{DATASET_NAME}_XXX.json`: 統合データセット
- `{DATASET_NAME}_XXX_sharegpt.jsonl`: ShareGPT形式
- `quality_analysis.json`: 品質分析結果
- `quality_analysis.png`: 可視化グラフ

### 🚀 次のステップ
1. **ファインチューニング**: 生成データでモデル訓練
2. **品質評価**: HLE試験問題での性能測定
3. **スケールアップ**: より大規模なデータセット生成
4. **ドメイン拡張**: 追加数学分野の対応

### 📚 参考資料
- [Magpie論文](https://arxiv.org/abs/2406.08464)
- [DeepSeek R1](https://huggingface.co/deepseek-ai/DeepSeek-R1)
- [プロジェクトGitHub](https://github.com/your-repo/magpie)

**🎯 Colab版6ドメイン数学データセット生成完了！**